<a href="https://colab.research.google.com/github/ujwalbholan/LinearRegression_model/blob/main/brainStrokeRegressionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
from google.colab import files

uploaded = files.upload()


Saving brain_stroke.csv to brain_stroke (1).csv


In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_predict, KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score,accuracy_score

In [51]:
stroke = pd.read_csv('brain_stroke.csv')

In [52]:
stroke.head(6)

gender   age  hypertension  heart_disease ever_married      work_type  \
0    Male  67.0             0              1          Yes        Private   
1    Male  80.0             0              1          Yes        Private   
2  Female  49.0             0              0          Yes        Private   
3  Female  79.0             1              0          Yes  Self-employed   
4    Male  81.0             0              0          Yes        Private   
5    Male  74.0             1              1          Yes        Private   

  Residence_type  avg_glucose_level   bmi   smoking_status  stroke  
0          Urban             228.69  36.6  formerly smoked       1  
1          Rural             105.92  32.5     never smoked       1  
2          Urban             171.23  34.4           smokes       1  
3          Rural             174.12  24.0     never smoked       1  
4          Urban             186.21  29.0  formerly smoked       1  
5          Rural              70.09  27.4     never smoked       1

In [53]:
numericlData = stroke.select_dtypes(include='number')

In [54]:
numericlData

age  hypertension  heart_disease  avg_glucose_level   bmi  stroke
0     67.0             0              1             228.69  36.6       1
1     80.0             0              1             105.92  32.5       1
2     49.0             0              0             171.23  34.4       1
3     79.0             1              0             174.12  24.0       1
4     81.0             0              0             186.21  29.0       1
...    ...           ...            ...                ...   ...     ...
4976  41.0             0              0              70.15  29.8       0
4977  40.0             0              0             191.15  31.1       0
4978  45.0             1              0              95.02  31.8       0
4979  40.0             0              0              83.94  30.0       0
4980  80.0             1              0              83.75  29.1       0

[4981 rows x 6 columns]

Here’s a complete linear regression example for your stroke dataset that:

 - Splits the data into train/test sets,

 - Standardizes features,

 - Trains a Linear Regression model,

 - Predicts continuous outputs,

 - Calculates MSE and R²,

Converts predictions into binary (threshold 0.5) for a rough classification accuracy.

# -- Note as i am making my model using linear regression it is using sigmodi as the output layer to pridict the output. hidden layers is relu .

In [55]:

X = numericlData[['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi']]
y = numericlData['stroke']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse:.4f}")
print(f"R² Score: {r2 * 100:.2f}%")

y_pred_binary = (y_pred >= 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy (threshold=0.5): {accuracy * 100:.2f}%")

# --- Cross-validation on full dataset ---

# Scale entire X for CV
X_scaled = scaler.fit_transform(X)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
y_pred_cv = cross_val_predict(model, X_scaled, y, cv=kf)

# Binary classification accuracy on CV predictions
y_pred_binary_cv = (y_pred_cv >= 0.5).astype(int)
accuracy_cv = accuracy_score(y, y_pred_binary_cv)
print(f"Cross-Validated Accuracy (threshold=0.5): {accuracy_cv * 100:.2f}%")


MSE: 0.0469
R² Score: 8.47%
Accuracy (threshold=0.5): 94.58%
Cross-Validated Accuracy (threshold=0.5): 95.02%


# Testing the model i  dummy **data**

In [56]:
dummy_data = np.array([
    [65, 0, 0, 180.5, 25.6],
    [40, 1, 0, 120.0, 30.1]
])

dummy_df = pd.DataFrame(dummy_data, columns=['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi'])

dummy_scaled = scaler.transform(dummy_df)

dummy_pred = model.predict(dummy_scaled)

print("Stroke risk scores:", dummy_pred)

dummy_pred_binary = (dummy_pred >= 0.5).astype(int)

for i, pred in enumerate(dummy_pred_binary, start=1):
    if pred == 1:
        print(f"Patient {i}: Predicted to have a stroke.")
    else:
        print(f"Patient {i}: Predicted NOT to have a stroke.")

Stroke risk scores: [0.121003   0.07979955]
Patient 1: Predicted NOT to have a stroke.
Patient 2: Predicted NOT to have a stroke.


In [57]:
import pickle


with open('linear_regression_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)